In [1]:
import os
os.chdir('..')

In [3]:
from jh_utils.utils.timer import Timer

In [4]:
## it’s mine, my own, 
from modules.get_data import get_x_y
from jh_utils.data.pandas.preprocessing import make_dummies
from modules.evaluation import fit_classify, print_2_reports, plot_false_negative_rate
from modules.models import multi_layer_model

2023-07-18 07:56:04.992375: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 07:56:05.790275: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold
## posfit
from sklearn.metrics import roc_auc_score, precision_recall_curve, det_curve
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
from sklearn.metrics import classification_report,confusion_matrix

In [6]:
## models
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

---

In [ ]:
def notes(thr, y_prob, y_test):
    y_pred = list(map(lambda x: x > thr, y_prob))
    cm = confusion_matrix(y_test, y_pred)
    nota = [cm[1,0],cm[0,1],sum(cm[1,:]), thr]
    return nota


def run_3_models(i):
    scaler = MinMaxScaler()
    x_train, x_test = x.iloc[i[0]].to_numpy(), x.iloc[i[1]].to_numpy()
    y_train, y_test = y.iloc[i[0]].to_numpy()[:,0], y.iloc[i[1]].to_numpy()[:,0]

    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    ### tf
    tf = multi_layer_model([16,16,10])
    callback = EarlyStopping(monitor='loss', patience=4)
    tf.fit(x_train, y_train, epochs=100, batch_size=128, verbose=0, callbacks=[callback])
    y_prob = tf.predict(x_test)
    save.append(notes(thr=0.5, y_prob=y_prob, y_test = y_test)+['TF'])
    save.append(notes(thr=0.05,y_prob=y_prob, y_test = y_test)+['TF'])
    print('tf')

In [7]:
def notes(thr, y_prob, y_test):
    y_pred = list(map(lambda x: x > thr, y_prob))
    cm = confusion_matrix(y_test, y_pred)
    nota = [cm[1,0],cm[0,1],sum(cm[1,:]), thr]
    return nota


def run_3_models(i):
    scaler = MinMaxScaler()
    x_train, x_test = x.iloc[i[0]].to_numpy(), x.iloc[i[1]].to_numpy()
    y_train, y_test = y.iloc[i[0]].to_numpy()[:,0], y.iloc[i[1]].to_numpy()[:,0]

    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    ### ada
    ada = AdaBoostClassifier()
    ada.fit(x_train, y_train)
    y_prob = ada.predict_proba(x_test)[:,1]
    save.append(notes(thr=0.5, y_prob=y_prob, y_test = y_test)+['ADA'])
    save.append(notes(thr=0.05, y_prob=y_prob, y_test = y_test)+['ADA'])
    print('ada')
    
    ### rf
    rf = RandomForestClassifier(50)
    rf.fit(x_train, y_train)
    y_prob = rf.predict_proba(x_test)[:,1]
    save.append(notes(thr=0.5, y_prob=y_prob, y_test = y_test)+['RF'])
    save.append(notes(thr=0.05, y_prob=y_prob, y_test = y_test)+['RF'])
    print('rf')
    
    ### tf
    tf = multi_layer_model([16,16,10])
    callback = EarlyStopping(monitor='loss', patience=4)
    tf.fit(x_train, y_train, epochs=100, batch_size=128, verbose=0, callbacks=[callback])
    y_prob = tf.predict(x_test)
    save.append(notes(thr=0.5, y_prob=y_prob, y_test = y_test)+['TF'])
    save.append(notes(thr=0.05,y_prob=y_prob, y_test = y_test)+['TF'])
    print('tf')

In [8]:
save = []
n_splits = 10
kfold = StratifiedKFold(n_splits=10,random_state=42,shuffle=True)
x,y = get_x_y()

/home/joaoholz/Documents/ufes/trabalho_final/modules/get_data.py:6: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv("asteroides/asteoides.csv").iloc[:, first_columns:]


In [9]:
for i in kfold.split(x,y):
    run_3_models(i)

ada
rf


2023-07-18 07:59:07.489948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-18 07:59:07.639388: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-18 07:59:07.640098: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

2914/2914 [==============================] - 3s 857us/step
tf
ada
rf
2914/2914 [==============================] - 3s 854us/step
tf
ada
rf
2914/2914 [==============================] - 2s 839us/step
tf
ada
rf
2914/2914 [==============================] - 3s 882us/step
tf
ada
rf
2914/2914 [==============================] - 2s 825us/step
tf
ada
rf
2914/2914 [==============================] - 2s 838us/step
tf
ada
rf
2914/2914 [==============================] - 2s 838us/step
tf
ada
rf
2914/2914 [==============================] - 3s 845us/step
tf
ada
rf
2914/2914 [==============================] - 3s 849us/step
tf
ada
rf
2914/2914 [==============================] - 3s 850us/step
tf


In [14]:
pd.DataFrame(save).to_csv('kfold2.csv',index=False)

In [13]:
pd.DataFrame(save)

,0,1,2,3,4
0,2,2,207,0.50,ADA
1,0,93026,207,0.05,ADA
2,2,2,207,0.50,RF
3,0,105,207,0.05,RF
4,52,9,207,0.50,TF
5,0,102,207,0.05,TF
6,3,5,207,0.50,ADA
7,0,93027,207,0.05,ADA
8,4,5,207,0.50,RF
9,0,155,207,0.05,RF


In [11]:
# save = []
# n_splits = 10
# kfold = StratifiedKFold(n_splits=10,random_state=42,shuffle=True)
# for i in kfold.split(x,y):
#     scaler = MinMaxScaler()
#     x_train, x_test = x.iloc[i[0]].to_numpy(), x.iloc[i[1]].to_numpy()
#     y_train, y_test = y.iloc[i[0]].to_numpy()[:,0], y.iloc[i[1]].to_numpy()[:,0]

#     x_train = scaler.fit_transform(x_train)
#     x_test = scaler.transform(x_test)
    
#     model = multi_layer_model([16,16,10])
#     model.fit(x_train, y_train, epochs=15, batch_size=128, verbose=1)
#     y_prob = model.predict(x_test)
#     thr = 0.05
#     y_pred = list(map(lambda x: x > thr, y_prob))
#     cm = confusion_matrix(y_test, y_pred)
#     nota = [cm[1,0],sum(cm[1,:]), thr]
#     print('0')
#     break